In [1]:
pip install pymongo

     ---------------------------------------- 0.0/472.7 kB ? eta -:--:--
     ------------------------------------- 472.7/472.7 kB 14.9 MB/s eta 0:00:00
  Using cached dnspython-2.4.2-py3-none-any.whl (300 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pymongo import MongoClient
import random

client = MongoClient('mongodb://localhost:27017/')
db = client['sample_db']
collection1_data = [
    {"_id": i, "name": f"Item{i}"} for i in range(10)
]
# Insert data into collection1
collection1 = db['collection1']
collection1.insert_many(collection1_data)
# Sample data for collection2
collection2_data = [
    {"id": random.choice(range(10)), "value2": random.randint(1, 100)} for _ in range(50)
]
# Insert data into collection2
collection2 = db['collection2']
collection2.insert_many(collection2_data)
print("Sample data has been inserted into 'collection1' and 'collection2'")

Sample data has been inserted into 'collection1' and 'collection2'


In [4]:
# Select the database
db = client['sample_db']

# Assuming 'id' is the name of the field you're matching on in both collections
# and 'value2' is the field you want to tally in 'collection2'
# Aggregation pipeline to match and sum
pipeline = [
    {
        '$lookup': {
            'from': 'collection2',  # Join with collection2
            'localField': '_id',  # Field from collection1
            'foreignField': 'id',  # Field from collection2 to match on
            'as': 'matched_docs'  # Output array with matched documents
        }
    },
    {
        '$unwind': '$matched_docs'  # Unwind the array of matched documents
    },
    {
        '$group': {
            '_id': '$_id',  # Group by the original document id
            'total_value2': {'$sum': '$matched_docs.value2'}  # Sum up the value2 from collection2
        }
    },
    {
        '$sort': {'total_value2': -1}  # Sort by total_value2 in descending order
    },
    {
        '$limit': 10  # Limit to top 10 results
    }
]
# Execute the aggregation pipeline
result = db['collection1'].aggregate(pipeline)
# Convert the aggregation result to a dictionary mapping id to value2
result_dict = {str(doc['_id']): doc['total_value2'] for doc in result}
print(result_dict)

{'5': 448, '4': 394, '6': 379, '2': 372, '8': 364, '0': 193, '3': 165, '1': 148, '7': 140, '9': 60}
